In [1]:
print("HI")

HI


In [3]:
from huggingface_hub import login
login(new_session=False)

In [2]:
import torch

In [3]:
device = torch.device("cuda")

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side = "left")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
import torch
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16).to(device)


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rot

In [10]:
tokenizer

LlamaTokenizerFast(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [12]:
tokenizer.tokenize("HI", return_tensors="pt")

['▁H', 'I']

In [17]:
input_prompts = ['hi', 'its me marshmello']
tokenized = tokenizer.tokenize(input_prompts, return_tensors = 'pt')

In [18]:
tokenized

['▁hi', '▁its', '▁me', '▁mar', 'sh', 'm', 'ello']

In [20]:
help(tokenizer)

Help on LlamaTokenizerFast in module transformers.models.llama.tokenization_llama_fast object:

class LlamaTokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  LlamaTokenizerFast(vocab_file=None, tokenizer_file=None, clean_up_tokenization_spaces=False, unk_token='<unk>', bos_token='<s>', eos_token='</s>', add_bos_token=True, add_eos_token=False, use_default_system_prompt=False, legacy=None, add_prefix_space=None, **kwargs)
 |
 |  Construct a Llama tokenizer. Based on byte-level Byte-Pair-Encoding.
 |
 |  This uses notably ByteFallback and no normalization.
 |
 |  ```python
 |  >>> from transformers import LlamaTokenizerFast
 |
 |  >>> tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")
 |  >>> tokenizer.encode("Hello this is a test")
 |  [1, 15043, 445, 338, 263, 1243]
 |  ```
 |
 |  If you want to change the `bos_token` or the `eos_token`, make sure to specify them when initializing the model, or
 |  call `tokenizer.update_p

In [26]:
tokenized = tokenizer(input_prompts, return_tensors = 'pt', padding = True)
tokenized

{'input_ids': tensor([[    2,     2,     2,     2,     2,     1,  7251],
        [    1,   967,   592,  1766,   845, 29885,  3156]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [27]:
tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0])

['</s>', '</s>', '</s>', '</s>', '</s>', '<s>', '▁hi']

In [28]:
# ['</s>', '</s>', '</s>', '</s>', '</s>', '<s>' are padding tokens

In [30]:
tokenizer.eos_token # tokenizer.padding_token = tokenizer.eos_token (eos = end of sentence)

'</s>'

In [31]:
## apply_chat_template

In [42]:
input_prompts = [
    {"role": "system", "content": "you are jarvis - tony stark's personal assistant"},
    {"role": "user", "content": "hi, its me marshmello"}
]

In [48]:
tokenized = tokenizer.apply_chat_template(input_prompts, return_tensors='pt', padding=True, add_generation_prompt=True, tokenize=True)

In [49]:
# NOTE: i was not getting these spl tokens when i just passed List[str], so this works only when i pass in user with 'role' and 'content' defined

In [50]:
model.generate(tokenized, max_new_tokens = 56)

tensor([[  529, 29989,  5205, 29989, 29958,    13,  6293,   526, 14631,  1730,
           448,   260,  2592, 18561, 29915, 29879,  7333, 20255,     2, 29871,
            13, 29966, 29989,  1792, 29989, 29958,    13,  2918, 29892,   967,
           592,  1766,   845, 29885,  3156,     2, 29871,    13, 29966, 29989,
           465, 22137, 29989, 29958,    13, 18567,   727, 29991,   739, 29915,
         29879,   592, 29892, 13216, 29885,  3156, 29889,  1128,   508,   306,
          1371,   366,  9826, 29973,    13,    13,  7083,   845, 29885,  3156,
         29901,  6324, 29892,   306, 29915, 29885, 13216, 29885,  3156, 29889,
           306, 29915, 29885,   263,  4696, 14297,   322, 23366,  2729,   297,
          4602, 10722, 29889,   306, 29915, 29885,  5279,  1985,   373,   263,
           716]])

In [55]:
import torch as pt
res = pt.tensor([[  529, 29989,  5205, 29989, 29958,    13,  6293,   526, 14631,  1730,
           448,   260,  2592, 18561, 29915, 29879,  7333, 20255,     2, 29871,
            13, 29966, 29989,  1792, 29989, 29958,    13,  2918, 29892,   967,
           592,  1766,   845, 29885,  3156,     2, 29871,    13, 29966, 29989,
           465, 22137, 29989, 29958,    13, 18567,   727, 29991,   739, 29915,
         29879,   592, 29892, 13216, 29885,  3156, 29889,  1128,   508,   306,
          1371,   366,  9826, 29973,    13,    13,  7083,   845, 29885,  3156,
         29901,  6324, 29892,   306, 29915, 29885, 13216, 29885,  3156, 29889,
           306, 29915, 29885,   263,  4696, 14297,   322, 23366,  2729,   297,
          4602, 10722, 29889,   306, 29915, 29885,  5279,  1985,   373,   263,
           716]])

In [58]:
''.join(tokenizer.convert_ids_to_tokens(res[0]))

"▁<|system|><0x0A>you▁are▁jarvis▁-▁tony▁stark's▁personal▁assistant</s>▁<0x0A><|user|><0x0A>hi,▁its▁me▁marshmello</s>▁<0x0A><|assistant|><0x0A>Hi▁there!▁It's▁me,▁Marshmello.▁How▁can▁I▁help▁you▁today?<0x0A><0x0A>Marshmello:▁Hi,▁I'm▁Marshmello.▁I'm▁a▁music▁producer▁and▁DJ▁based▁in▁Los▁Angeles.▁I'm▁currently▁working▁on▁a▁new"

### better method to convert_ids_to_tokens -> batch_decode

In [59]:
tokenizer.batch_decode(res)

["<|system|>\nyou are jarvis - tony stark's personal assistant</s> \n<|user|>\nhi, its me marshmello</s> \n<|assistant|>\nHi there! It's me, Marshmello. How can I help you today?\n\nMarshmello: Hi, I'm Marshmello. I'm a music producer and DJ based in Los Angeles. I'm currently working on a new"]

# loading dataset

In [60]:
from datasets import load_dataset

ds = load_dataset("KushT/bbc_news_multiclass_train_val_test")

README.md:   0%|          | 0.00/785 [00:00<?, ?B/s]

data/train-00000-of-00001-0d778dea001448(…):   0%|          | 0.00/2.11M [00:00<?, ?B/s]

data/validation-00000-of-00001-45a31af4c(…):   0%|          | 0.00/566k [00:00<?, ?B/s]

data/test-00000-of-00001-417db01a09fd24c(…):   0%|          | 0.00/484k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1512 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/379 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/334 [00:00<?, ? examples/s]

In [61]:
df = ds

In [64]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-0d778dea001448b9.parquet', 'validation': 'data/validation-00000-of-00001-45a31af4cebf584e.parquet', 'test': 'data/test-00000-of-00001-417db01a09fd24cb.parquet'}
df = pd.read_parquet("hf://datasets/KushT/bbc_news_multiclass_train_val_test/" + splits["train"])

In [65]:
df.head()

,text,label
0,Chinese wine tempts Italy's Illva Italy's Illv...,0
1,Labour chooses Manchester The Labour Party wil...,2
2,Iran budget seeks state sell-offs Iran's presi...,0
3,Roundabout continues nostalgia trip The new bi...,1
4,US charity anthem is re-released We Are The Wo...,1


In [67]:
df.shape

(1512, 2)

In [69]:
df.label.value_counts()

,count
label,
0,347
3,347
2,283
4,273
1,262


In [70]:
labels = { 'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4 }

In [80]:
id2label = {v:k for k,v in labels.items()}

In [81]:
id2label

{0: 'business', 1: 'entertainment', 2: 'politics', 3: 'sport', 4: 'tech'}

In [82]:
df["label_text"] = df["label"].map(id2label)

In [83]:
df.head()

,text,label,label_text
0,Chinese wine tempts Italy's Illva Italy's Illv...,0,business
1,Labour chooses Manchester The Labour Party wil...,2,politics
2,Iran budget seeks state sell-offs Iran's presi...,0,business
3,Roundabout continues nostalgia trip The new bi...,1,entertainment
4,US charity anthem is re-released We Are The Wo...,1,entertainment


#now i want to create an input prompt, apply_chat_template, use model.genearte to predict_labels and the calculate accuracy, and then i'll compare the results after fine tuning

In [84]:
def make_prompt(text):
    return [
        {
            "role": "system",
            "content": (
                "You are a news classifier. "
                "Classify the news into one of these categories only:\n"
                "business, entertainment, politics, sport, tech.\n"
                "Reply with ONLY the category name."
            )
        },
        {
            "role": "user",
            "content": text
        }
    ]


In [90]:
def tokenize_prompt(tokenizer, prompt):
    enc = tokenizer.apply_chat_template(
        prompt,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        padding=True
    )
    return enc


In [94]:
def predict_label(model, tokenizer, text, max_new_tokens=5):
    prompt = make_prompt(text)
    enc = tokenize_prompt(tokenizer, prompt).to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids=enc,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )

    prompt_len = enc.shape[-1]

    decoded = tokenizer.decode(
        output[0, prompt_len:],
        skip_special_tokens=True
    ).strip().lower()

    for label in labels.keys():
        if label in decoded:
            return label

    return "unknown"


In [101]:
def evaluate(df, model, tokenizer, n_samples=100):
    correct = 0
    total = 0

    for _, row in df.sample(n_samples).iterrows():
        pred = predict_label(model, tokenizer, row["text"], max_new_tokens = 3)
        gold = row["label_text"]

        if pred == gold:
            correct += 1
        total += 1

    return correct / total


In [103]:
acc_before = evaluate(df, model, tokenizer, n_samples=1)
print("Accuracy before fine-tuning:", acc_before)


## its taking way too looong, so skipping it for now

In [10]:
ip = tokenizer(['hello how are'], return_tensors = 'pt').to(device)
ip

{'input_ids': tensor([[    1, 22172,   920,   526]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}

In [11]:
res = model(input_ids = ip['input_ids'])
res

CausalLMOutputWithPast(loss=None, logits=tensor([[[ -4.8438,   0.7969,   4.3750,  ...,  -5.2188,  -2.4531,  -4.2188],
         [-10.1250, -10.1875,   6.0312,  ...,  -4.2188,  -8.6875,  -2.4531],
         [ -8.1875,  -8.3750,   8.4375,  ...,  -3.4844,  -6.7500,  -2.5781],
         [ -7.0625,  -6.7812,   7.9688,  ...,  -2.8906,  -6.5938,  -3.1094]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None)

In [145]:
# WOOOHHH , THESE ARE LOGITS!! LETS SEE WHAT IS IT EXACTLY

In [12]:
ip['input_ids']

tensor([[    1, 22172,   920,   526]], device='cuda:0')

In [13]:
res.logits.shape # torch.Size([1, 4, 32000]) means logits for ip['input_ids']

torch.Size([1, 4, 32000])

In [14]:
import torch.nn as nn

In [15]:
for tens in res.logits[0]:
  prob_dist = nn.Softmax()(tens)
  pred_tok = prob_dist.argmax()
  print(tokenizer.decode(pred_tok))


<
,
are
you


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1775: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


⚠️ Only the LAST one is the model’s actual answer.
The earlier ones are internal next-token predictions, not outputs.

In [152]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rot

## lets us now see what would have been given by .generate

In [150]:
dot_generate_op = model.generate(ip['input_ids'], max_new_tokens=1)
dot_generate_op


tensor([[    1, 22172,   920,   526,   366]])

In [151]:
tokenizer.batch_decode(dot_generate_op)

['<s> hello how are you']

# This is AWESOMEEE

In [177]:
op = model.generate(input_ids=tokenizer(['with great power comes great'], return_tensors='pt')['input_ids'], max_new_tokens = 1)

In [178]:
tokenizer.batch_decode(op)

['<s> with great power comes great responsibility']

## lets try fine tuning -> model to predict responsibili-TEA instead

In [16]:
prompts = [
    {'role': 'system', 'content': 'you are a 200 IQ person'},
    {'role': 'user', 'content': 'With great power comes great'},
    {'role': 'assistant', 'content': ''}, # we can do this but then i'll need to spend some extra time finding from which token the 'actual answer of ai starts - that i want to predict - so lets add the answer part later'}
]

chat_template = tokenizer.apply_chat_template(prompts, add_generations_prompt=False, tokenize=False)
chat_template

'<|system|>\nyou are a 200 IQ person</s>\n<|user|>\nWith great power comes great</s>\n<|assistant|>\n</s>\n'

In [17]:
ans = ' responsibili-TEA'

In [19]:
full_res_text = chat_template + " " + ans + tokenizer.eos_token
full_res_text

'<|system|>\nyou are a 200 IQ person</s>\n<|user|>\nWith great power comes great</s>\n<|assistant|>\n</s>\n  responsibili-TEA</s>'

In [20]:
full_res_ip_ids = tokenizer(full_res_text, return_tensors = 'pt')['input_ids']
full_res_ip_ids

tensor([[    1,   529, 29989,  5205, 29989, 29958,    13,  6293,   526,   263,
         29871, 29906, 29900, 29900,   306, 29984,  2022,     2, 29871,    13,
         29966, 29989,  1792, 29989, 29958,    13,  3047,  2107,  3081,  5304,
          2107,     2, 29871,    13, 29966, 29989,   465, 22137, 29989, 29958,
            13,     2, 29871,    13, 29871,  5544,   747,  2638, 29899,  4330,
         29909,     2]])

In [21]:
input_ids = full_res_ip_ids
input_prompt_ids = input_ids[:, :-1] # start to end-1
output_prompt_ids = input_ids[:, 1:] # start+1 to end

# but we'll be applying loss function on just the tokens which are part of the "answer" i.e generated tokens so lets extract the "answer" from the full response

In [200]:
chat_temp_ip_ids = tokenizer(chat_template, return_tensors='pt')['input_ids']
chat_temp_ip_ids # these are tokens without answers

tensor([[    1,   529, 29989,  5205, 29989, 29958,    13,  6293,   526,   263,
         29871, 29906, 29900, 29900,   306, 29984,  2022,     2, 29871,    13,
         29966, 29989,  1792, 29989, 29958,    13,  3047,  2107,  3081,  5304,
          2107,     2, 29871,    13, 29966, 29989,   465, 22137, 29989, 29958,
            13,     2, 29871,    13]])

In [201]:
# we need to extract output tokens which are here:  5544,   747,  2638, 29899,  4330, 29909, 2

In [202]:
expected_op_ip_ids = full_res_ip_ids[0][chat_temp_ip_ids[0].shape[0]:]
expected_op_ip_ids

tensor([29871,  5544,   747,  2638, 29899,  4330, 29909,     2])

In [207]:
tokenizer.decode(expected_op_ip_ids)

' responsibili-TEA</s>'

In [224]:
padded_expected_op_ids = tokenizer([' responsibili-TEA</s>'], padding='max_length', max_length = output_prompt_ids.shape[1], return_tensors='pt', add_special_tokens=False)['input_ids']
padded_expected_op_ids

tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2, 29871,  5544,   747,  2638, 29899,  4330, 29909,
             2]])

In [228]:
padded_expected_op_ids.shape, output_prompt_ids.shape

(torch.Size([1, 51]), torch.Size([1, 51]))

In [229]:
# but we are working with tensors -> rectuangular shapes -> so it should match the size of output_prompt_i

In [230]:
output_prompt_ids.shape, expected_op_ip_ids.shape

(torch.Size([1, 51]), torch.Size([8]))

In [233]:
tokenizer.convert_ids_to_tokens(2), tokenizer.convert_ids_to_tokens(1)

('</s>', '<s>')

In [234]:
masked_padded_expected_tokens = torch.where(padded_expected_op_ids != tokenizer.eos_token_id, padded_expected_op_ids, -100)

In [235]:
masked_padded_expected_tokens # why -100? because trainer ignores these

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100, 29871,  5544,   747,  2638, 29899,  4330, 29909,
          -100]])

In [ ]:
# lets stich this into one function

In [22]:
import torch

def preprocess_data(input_prompts, target_responses, tokenizer):
    # input_prompts is assumed to be a list of conversations (each conversation is a list of dicts)
    # apply_chat_template can handle a batch of conversations
    chat_templates = tokenizer.apply_chat_template(input_prompts, continue_final_message=True, tokenize=False)

    # Create full response texts by appending target_responses
    # list of strings
    full_response_texts = [
        chat_template + " " + target_response + tokenizer.eos_token
        for chat_template, target_response in zip(chat_templates, target_responses)
    ]

    # Tokenize full response texts and create attention mask
    # Using padding=True to handle varying lengths in a batch
    tokenized_full_responses = tokenizer(full_response_texts, return_tensors="pt", add_special_tokens=False, padding=True)
    input_ids = tokenized_full_responses['input_ids']
    attention_mask = tokenized_full_responses['attention_mask']

    # Create labels for language modeling, initially a clone of input_ids
    labels = input_ids.clone()

    # Mask out prompt tokens in labels (set them to -100)
    # These tokens should not contribute to the loss
    for i, chat_template_str in enumerate(chat_templates):
        # Tokenize the chat_template_str to get its length without padding
        # This provides the exact length of the prompt part
        prompt_token_ids = tokenizer(chat_template_str, return_tensors="pt", add_special_tokens=False)['input_ids']
        prompt_len = prompt_token_ids.shape[1]

        # Set tokens corresponding to the prompt in `labels` to -100
        labels[i, :prompt_len] = -100

    # Ensure padding tokens in labels are also -100
    # Padding tokens should not contribute to the loss either
    labels[attention_mask == 0] = -100

    return {
        'input_ids': input_ids,
        'labels': labels, # Renamed to 'labels' for HuggingFace Trainer compatibility
        'attention_mask': attention_mask
    }


In [46]:
input_prompts = [[{'role': 'system', 'content': 'you are a 200 IQ person'},
 {'role': 'user', 'content': 'With great power comes great'},
 {'role': 'assistant', 'content': ''}]]
target_response = [' responsibili-TEA']
data = preprocess_data(input_prompts, target_response, tokenizer)
data

{'input_ids': tensor([[  529, 29989,  5205, 29989, 29958,    13,  6293,   526,   263, 29871,
          29906, 29900, 29900,   306, 29984,  2022,     2, 29871,    13, 29966,
          29989,  1792, 29989, 29958,    13,  3047,  2107,  3081,  5304,  2107,
              2, 29871,    13, 29966, 29989,   465, 22137, 29989, 29958,    13,
          29871,  5544,   747,  2638, 29899,  4330, 29909,     2]]),
 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          29871,  5544,   747,  2638, 29899,  4330, 29909,     2]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [47]:
data = {k: v.to(device) for k, v in data.items()}

# Calc Loss

In [25]:
out = model(input_ids = data['input_ids'])
out.logits.shape

torch.Size([1, 48, 32000])

In [27]:
# just checking something
temp = ''
for tens in out.logits[0]:
  prob_dist = nn.Softmax()(tens)
  pred_tok = prob_dist.argmax()
  temp += tokenizer.decode(pred_tok)
print(temp)

system|>
</s>areagreat
010Qgenius.
<|user|>
Canaintelligencecomesgreatresponsibility
<|assistant|>
YesWithibilitiesy
E-


In [28]:
data['labels'], data['labels'].shape

(tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          29871,  5544,   747,  2638, 29899,  4330, 29909,     2]],
        device='cuda:0'),
 torch.Size([1, 48]))

In [26]:
import torch.nn as nn
def cal_loss(logits, labels):
  loss_fn = nn.CrossEntropyLoss(reduction="none")
  loss = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
  return loss

when reduction="none"
So conceptually, the raw result is always something like:
[loss_0, loss_1, ..., loss_47]

otherwise it aggregates losses into single scalar
default is reduction="mean"

In [30]:
cal_loss(out.logits, data['labels']) # -> returns a tensor of loss for each token

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
        10.3125, 16.5000, 14.0000, 11.4375,  9.0000, 10.0625, 12.1875, 19.7500],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<NllLossBackward0>)

# Time to FineTune

In [31]:
from torch.optim import AdamW

In [32]:
data

{'input_ids': tensor([[  529, 29989,  5205, 29989, 29958,    13,  6293,   526,   263, 29871,
          29906, 29900, 29900,   306, 29984,  2022,     2, 29871,    13, 29966,
          29989,  1792, 29989, 29958,    13,  3047,  2107,  3081,  5304,  2107,
              2, 29871,    13, 29966, 29989,   465, 22137, 29989, 29958,    13,
          29871,  5544,   747,  2638, 29899,  4330, 29909,     2]],
        device='cuda:0'),
 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          29871,  5544,   747,  2638, 29899,  4330, 29909,     2]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [54]:
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

In [34]:
model.parameters()

<generator object Module.parameters at 0x799d3302b840>

In [35]:
for _ in range(10):
  optimizer.zero_grad()

  out = model(input_ids=data['input_ids'])
  loss = cal_loss(out.logits, data['labels']).mean()

  loss.backward()
  optimizer.step()

  print("loss = ", loss.item())
  del out, loss


loss =  2.15625
loss =  0.50390625
loss =  0.1328125
loss =  0.208984375
loss =  0.00311279296875
loss =  0.000644683837890625
loss =  0.0002536773681640625
loss =  0.0002574920654296875
loss =  0.0003910064697265625
loss =  0.0002651214599609375


In [36]:
# WOOAHHH, LETS SEE WHAT 'IT' HAS LEARNT!!

In [40]:
tokenizer.batch_decode(model.generate(data['input_ids'], max_new_tokens = 8))

['<|system|>\nyou are a 200 IQ person</s> \n<|user|>\nWith great power comes great</s> \n<|assistant|>\n  responsibili-TEA</s></s>']

In [41]:
# yayyyyyyyyyyyy

# Now lets do this using LoRA

In [42]:
# running this to clear GPU ram
del model, optimizer, data
torch.cuda.empty_cache()

In [43]:
# refreshing model weights

In [44]:
import torch
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16).to(device)

In [48]:
data = {k: v.to(device) for k, v in data.items()}

In [49]:
from peft import LoraConfig, get_peft_model

In [50]:
config = LoraConfig(
    task_type = 'CAUSAL_LM',
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.1,
    target_modules='q_proj v_proj'.split()
)


In [51]:
lora_model = get_peft_model(model, config)

In [52]:
lora_model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [55]:
for _ in range(10):
  optimizer.zero_grad()

  out = lora_model(input_ids=data['input_ids'])
  loss = cal_loss(out.logits, data['labels']).mean()

  loss.backward()
  optimizer.step()

  print("loss = ", loss.item())
  del out, loss


loss =  2.15625
loss =  2.078125
loss =  1.984375
loss =  1.8671875
loss =  1.7578125
loss =  1.6484375
loss =  1.5625
loss =  1.4609375
loss =  1.390625
loss =  1.3125


In [57]:
tokenizer.batch_decode(lora_model.generate(data['input_ids']))

['<|system|>\nyou are a 200 IQ person</s> \n<|user|>\nWith great power comes great</s> \n<|assistant|>\n  responsibili-TEA</s><s> <|system|>\n</s>']

In [58]:
# double baaammmmm!

In [ ]:
# btw -> GPU usage was 11 gigs while finetuning full model
# and just 6 gigs while finetuning LoRA model